In [1]:
#####Item based Matrix Factorization model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import itertools
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k)).todense()   #スパース行列の設定
    return Z

In [3]:
####データの発生####
##データの設定
k = 10   #基底数
vec_k = np.repeat(1, k)
item = 25000
publisher = 3000
tag = 200
N = 1000000   #総レコード数

In [4]:
##アイテムの共起を生成
#トピック割当を生成
topic = 35
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(1.5, topic), 1).reshape(-1)
z = np.dot(np.random.multinomial(1, theta, N), range(topic))

In [5]:
#多項分布からアイテムを生成
n = 10000
item_vec = np.array(range(item))
no_vec = np.array(range(n))
index = np.array(range(N)).reshape(int(N/n), n)
item_id1 = np.repeat(0, N)
item_id2 = np.repeat(0, N)
for i in range(int(N/n)):
    print(i)
    phi_vec = phi[z[index[i]], :]
    item1 = np.array(rmnom(phi_vec, n, item, no_vec))
    item2 = rmnom((1-item1)*phi_vec, n, item, no_vec)
    item_id1[index[i]] = np.dot(item1, item_vec)
    item_id2[index[i]] = np.dot(item2, item_vec)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
##パブリッシャーの割当を生成
#トピック割当を生成
topic = 30
phi = np.random.dirichlet(np.repeat(0.5, publisher), topic)
theta = np.random.dirichlet(np.repeat(0.25, topic), item)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(item)]).reshape(item, topic), range(topic))

#多項分布からパブリッシャーを生成
publisher_id = np.array(np.dot(rmnom(phi[z, :], item, publisher, np.array(range(item))), np.array(range(publisher)))).reshape(-1)
publisher_id1 = publisher_id[item_id1]
publisher_id2 = publisher_id[item_id2]

In [7]:
##タグの割当を生成
#トピック割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, tag), topic)
theta = np.random.dirichlet(np.repeat(0.25, topic), item)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(item)]).reshape(item, topic), range(topic))

#多項分布からパブリッシャーを生成
tag_id = np.array(np.dot(rmnom(phi[z, :], item, tag, np.array(range(item))), np.array(range(tag)))).reshape(-1)
tag_id1 = tag_id[item_id1]
tag_id2 = tag_id[item_id2]

In [8]:
##素性ベクトルを生成
k1 = 5; k2 = 6; k3 = 5
x1 = np.array(np.random.random(N*k1)).reshape((N, k1))
x2 = np.zeros((N, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, N)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), N)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
X = np.concatenate((np.repeat(1, N)[:, np.newaxis], x1, x2, x3), axis=1)
column = X.shape[1]

In [9]:
##パブリッシャーの説明変数を生成
k1 = 3; k2 = 4; k3 = 4
v1 = np.array(np.random.random(publisher*k1)).reshape((publisher, k1))
v2 = np.zeros((publisher, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    v2[:, j] = np.random.binomial(1, prob, publisher)
v3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), publisher)
v3 = np.delete(v3, np.argmin(np.sum(v3, axis=0)), axis=1)   #冗長な変数の削除
v = np.concatenate((np.repeat(1, publisher)[:, np.newaxis], v1, v2, v3), axis=1)
column_v = v.shape[1]

In [10]:
####応答変数を生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##パラメータを生成
    #パブリッシャーの階層モデルのパラメータを生成
    alpha_v = np.random.multivariate_normal(np.repeat(0, k), np.diag(np.repeat(0.15, k)), column_v); alpha_vt = alpha_v
    Cov_v = np.diag(np.repeat(0.025, k)); Cov_vt = Cov_v
    inv_Cov_v = np.linalg.inv(Cov_v)

    #タグの階層モデルのパラメータを生成
    alpha_g = np.repeat(0, k); alpha_gt = alpha_g
    Cov_g = np.diag(np.repeat(0.15, k)); Cov_gt = Cov_g
    inv_Cov_g = np.linalg.inv(Cov_g)

    #モデルパラメータの生成
    Sigma = 1; Sigmat = Sigma
    beta = np.append(-1.3, np.random.normal(0, 0.75, column-1))
    betat = beta

    #特徴行列のパラメータを生成
    theta_v = np.dot(v, alpha_v) + np.random.multivariate_normal(np.repeat(0, k), Cov_v, publisher)
    theta_g = np.random.multivariate_normal(alpha_g, Cov_g, tag)
    theta_vt = theta_v; theta_gt = theta_g

    ##効用関数から応答変数を生成
    #行列分解のパラメータを設定
    WH1 = np.dot(theta_v[publisher_id1] * theta_v[publisher_id2], vec_k)
    WH2 = np.dot(theta_g[tag_id1] * theta_g[tag_id2], vec_k)

    #潜在効用を設定
    mu = np.dot(X, beta) + WH1 + WH2
    U = mu + np.random.normal(0, Sigma, N)

    #応答変数を変換
    y = np.array(U > 0, dtype="int")
    
    if((np.mean(y) > 0.2) & (np.mean(y) < 0.4)):
        break

1
2
3
4
5
6
7
8
9
10


In [11]:
##インデックスの設定
#パブリッシャーのインデックス
index = np.array(range(N))
publisher_list = [j for j in range(publisher)]
v_id = [j for j in range(publisher)]
for j in range(publisher):
    index1 = index[publisher_id1==j]
    index2 = index[publisher_id2==j]
    publisher_list[j] = np.append(index1, index2)
    v_id[j] = np.append(publisher_id2[index1], publisher_id1[index2])
    
#タグのインデックス
tag_list = [j for j in range(tag)]
g_id = [j for j in range(tag)]
for j in range(tag):
    index1 = index[tag_id1==j]
    index2 = index[tag_id2==j]
    tag_list[j] = np.append(index1, index2)
    g_id[j] = np.append(tag_id2[index1], tag_id1[index2])

In [12]:
####マルコフ連鎖モンテカルロ法でItem based Matrix Factorization modelを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [13]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [14]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0 
disp =10

In [15]:
##事前分布の設定
#素性ベクトルの事前分布
gamma = np.repeat(0, column)
inv_tau = np.linalg.inv(np.diag(np.repeat(100, column)))

#階層モデルの事前分布
Deltabar = np.zeros((column_v, k))
ADelta =0.01 * np.diag(np.ones(column_v))
nu = k + 1
V = np.diag(np.ones(k))

In [16]:
##パラメータの真値
#階層モデルのパラメータの真値
alpha_v = alpha_vt
alpha_g = alpha_gt
Cov_v = Cov_vt
Cov_g = Cov_gt
v_mu = np.dot(v, alpha_v)
inv_Cov_v = np.linalg.inv(Cov_v)
inv_Cov_g = np.linalg.inv(Cov_g)

#モデルパラメータの真値
Sigma = Sigmat
beta = betat
beta_mu = np.dot(X, beta)

#特徴行列と行列分解のパラメータの真値
theta_v = theta_vt
theta_g = theta_gt
WH1 = np.dot(theta_v[publisher_id1] * theta_v[publisher_id2], vec_k)
WH2 = np.dot(theta_g[tag_id1] * theta_g[tag_id2], vec_k)

In [17]:
##パラメータの初期値
#階層モデルのパラメータの初期値
alpha_v = np.zeros((column_v, k))
alpha_g = np.repeat(0, k)
Cov_v = np.diag(np.repeat(0.3, k))
Cov_g = np.diag(np.repeat(0.3, k))
v_mu = np.dot(v, alpha_v)
inv_Cov_v = np.linalg.inv(Cov_v)
inv_Cov_g = np.linalg.inv(Cov_g)

#モデルパラメータの初期値
Sigma = 1
beta = np.append(-1.0, np.repeat(0, column-1))
beta_mu = np.dot(X, beta)

#特徴行列と行列分解のパラメータの初期値
theta_v = v_mu + np.random.multivariate_normal(np.repeat(0, k), Cov_v, publisher)
theta_g = np.random.multivariate_normal(alpha_g, Cov_g, tag)
WH1 = np.dot(theta_v[publisher_id1] * theta_v[publisher_id2], vec_k)
WH2 = np.dot(theta_g[tag_id1] * theta_g[tag_id2], vec_k)

In [18]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

#定数を設定
XXU = np.dot(X.T, X) + inv_tau
inv_XXU = np.linalg.inv(XXU)

#階層モデルの定数
vv = np.dot(v.T, v)
inv_vv = np.linalg.inv(np.dot(v.T, v) + ADelta)

In [19]:
##サンプリングの格納用配列
BETA = np.zeros((int(R/keep), column))
THETA_V = np.zeros((publisher, k, int(R/keep)))
THETA_G = np.zeros((tag, k, int(R/keep)))
ALPHA_V = np.zeros((column_v, k, int(R/keep)))
COV_V = np.zeros((k, k, int(R/keep)))
COV_G = np.zeros((k, k, int(R/keep)))

In [20]:
##対数尤度の基準値
#1パラメータでの対数尤度
LLst = np.sum(y*np.log(np.mean(y))) + np.sum((1-y)*np.log(np.mean(1-y)))
print(LLst)

#真値での対数尤度
WHT1 = np.dot(theta_vt[publisher_id1] * theta_vt[publisher_id2], vec_k)
WHT2 = np.dot(theta_gt[tag_id1] * theta_gt[tag_id2], vec_k)
mu = np.dot(X, betat) + WHT1 + WHT2
Prob = norm.cdf(mu, 0, Sigma)
Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
print(LLbest)

-515874.1188896659
-213830.47011255886


In [27]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##切断正規分布から潜在効用を生成
    mu = beta_mu + WH1 + WH2   #モデルの平均構造 
    U = rtnorm(mu, Sigma, a, b, N)   #潜在効用を生成

    ##素性ベクトルの回帰ベクトルをサンプリング
    #モデル誤差を設定
    u_er = U - WH1 - WH2

    #多変量正規分布の事後分布を設定
    XU = np.dot(X.T, u_er)
    par_mu = np.dot(inv_XXU, XU)

    #多変量正規分布からパラメータをサンプリング
    beta = np.random.multivariate_normal(par_mu, inv_XXU, 1).reshape(-1)
    beta_mu = np.dot(X, beta)


    ##パブリッシャーの特徴行列をサンプリング
    #モデル誤差の設定
    u_er = U - beta_mu - WH2

    for j in range(publisher):
        #多変量正規分布の事後分布を設定
        index = publisher_list[j]
        x = theta_v[v_id[j], ]
        inv_XXV = np.linalg.inv(np.dot(x.T, x) + inv_Cov_v)
        Xy = np.dot(x.T, u_er[index])
        par_mu = np.dot(inv_XXV, Xy + np.dot(inv_Cov_v, v_mu[j, ]))

        #多変量正規分布から特徴ベクトルをサンプリング
        theta_v[j, ] = np.random.multivariate_normal(par_mu, inv_XXV, 1).reshape(-1)

    #特徴行列を更新
    WH1 = np.dot(theta_v[publisher_id1] * theta_v[publisher_id2], vec_k)

    ##タグの特徴行列をサンプリング
    #モデル誤差の設定
    u_er = U - beta_mu - WH1

    for j in range(tag):
        #多変量正規分布の事後分布を設定
        index = tag_list[j]
        x = theta_vt[g_id[j], ]
        inv_XXV = np.linalg.inv(np.dot(x.T, x) + inv_Cov_g)
        Xy = np.dot(x.T, u_er[index])
        par_mu = np.dot(inv_XXV, Xy + np.dot(inv_Cov_g, alpha_g))

        #多変量正規分布から特徴ベクトルをサンプリング
        theta_g[j, ] = np.random.multivariate_normal(par_mu, inv_XXV, 1).reshape(-1)

    #特徴行列を更新
    WH2 = np.dot(theta_g[tag_id1] * theta_g[tag_id2], vec_k)


    ##階層モデルのパラメータをサンプリング
    #多変量回帰モデルからパブリッシャーの特徴行列の階層モデルのパラメータをサンプリング
    out = rmultireg(theta_v, v, inv_vv, np.dot(v.T, theta_v), Cov_v, ADelta, Deltabar, V, nu, publisher, column_v, k)
    alpha_v = out[0]
    v_mu = np.dot(v, alpha_v)
    Cov_v = np.diag(np.diag(out[1]))
    inv_Cov_v = np.linalg.inv(Cov_v)

    #逆ウィシャート分布からタグの特徴行列の分散共分散行列をサンプリング
    er = theta_g - np.repeat(np.mean(theta_g, axis=0), tag).reshape(tag, k, order="F")
    IW = np.dot(er.T, er) + V
    Sn = tag + nu 
    Cov_g = np.diag(np.diag(scipy.stats.invwishart.rvs(Sn, IW, 1)))
    inv_Cov_g = np.linalg.inv(Cov_g)


    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep, ] = beta
        THETA_V[:, :, mkeep] = theta_v
        THETA_G[:, :, mkeep] = theta_g
        ALPHA_V[:, :, mkeep] = alpha_v
        COV_V[:, :, mkeep] = Cov_v
        COV_G[:, :, mkeep] = Cov_g

    if rp%disp==0:
        #対数尤度を計算
        mu = beta_mu + WHT1 + WHT2
        Prob = norm.cdf(mu, 0, Sigma)
        Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
        LL = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))

        #サンプリング結果を表示
        print(rp)
        print(np.round(np.array((LL, LLbest, LLst)), 1))
        print(np.round(np.vstack((beta, betat)), 3))
        print(np.round(np.vstack((np.diag(Cov_v), np.diag(Cov_vt))), 3))

0
[-223877.3 -213830.5 -515874.1]
[[-1.149 -0.157 -0.699 -0.897  0.149 -0.058  0.139  0.111  0.156 -0.191
  -1.176 -0.305  0.837 -0.833 -0.009 -0.917]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.058 0.027 0.025 0.022 0.021 0.025 0.026 0.02  0.031 0.024]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
10
[-223928.8 -213830.5 -515874.1]
[[-1.15  -0.17  -0.699 -0.892  0.147 -0.059  0.135  0.112  0.154 -0.196
  -1.176 -0.3    0.842 -0.824 -0.004 -0.905]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.062 0.028 0.023 0.023 0.021 0.021 0.025 0.022 0.035 0.024]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
20
[-224003.7 -213830.5 -515874.1]
[[-1.144 -0.175 -0.698 -0.883  0.157 -0.069  0.141  0.107  0.151 -0.195
  -1.179 -0.3    0.838 -0.828 -0.003 -0.911]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.1

200
[-224497.7 -213830.5 -515874.1]
[[-1.142e+00 -1.690e-01 -7.000e-01 -8.950e-01  1.520e-01 -6.400e-02
   1.380e-01  1.150e-01  1.520e-01 -1.950e-01 -1.172e+00 -2.970e-01
   8.350e-01 -8.240e-01 -1.000e-03 -9.080e-01]
 [-1.300e+00 -1.840e-01 -7.750e-01 -9.810e-01  1.700e-01 -7.100e-02
   1.460e-01  1.290e-01  1.710e-01 -2.210e-01 -1.305e+00 -3.380e-01
   9.340e-01 -9.210e-01 -2.300e-02 -1.010e+00]]
[[0.054 0.03  0.026 0.023 0.024 0.024 0.026 0.021 0.042 0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
210
[-224566.1 -213830.5 -515874.1]
[[-1.15  -0.171 -0.689 -0.885  0.144 -0.065  0.123  0.119  0.158 -0.198
  -1.176 -0.288  0.843 -0.818 -0.008 -0.898]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.066 0.027 0.022 0.023 0.022 0.023 0.025 0.023 0.045 0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
220
[-224818.5 -213830.5 -515874.1]
[[-1.133 -0.168 -0.694 -0.894  0.153

400
[-226447.4 -213830.5 -515874.1]
[[-1.118 -0.159 -0.695 -0.899  0.154 -0.063  0.135  0.116  0.148 -0.198
  -1.172 -0.306  0.847 -0.821 -0.008 -0.908]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.045 0.022 0.023 0.022 0.021 0.023 0.025 0.023 0.045 0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
410
[-226325.  -213830.5 -515874.1]
[[-1.129e+00 -1.610e-01 -7.030e-01 -8.890e-01  1.620e-01 -6.400e-02
   1.380e-01  1.210e-01  1.630e-01 -1.970e-01 -1.180e+00 -3.070e-01
   8.440e-01 -8.240e-01  1.000e-03 -8.970e-01]
 [-1.300e+00 -1.840e-01 -7.750e-01 -9.810e-01  1.700e-01 -7.100e-02
   1.460e-01  1.290e-01  1.710e-01 -2.210e-01 -1.305e+00 -3.380e-01
   9.340e-01 -9.210e-01 -2.300e-02 -1.010e+00]]
[[0.04  0.025 0.022 0.024 0.021 0.025 0.026 0.024 0.046 0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
420
[-226470.7 -213830.5 -515874.1]
[[-1.103 -0.167 -0.703 -0.883  0.142

590
[-227078.6 -213830.5 -515874.1]
[[-1.101 -0.181 -0.701 -0.89   0.137 -0.057  0.131  0.121  0.15  -0.202
  -1.171 -0.306  0.848 -0.815  0.    -0.898]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.052 0.022 0.024 0.024 0.02  0.021 0.029 0.022 0.032 0.022]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
600
[-227214.7 -213830.5 -515874.1]
[[-1.116 -0.166 -0.686 -0.879  0.142 -0.063  0.137  0.118  0.151 -0.195
  -1.177 -0.301  0.839 -0.827 -0.006 -0.896]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.053 0.023 0.022 0.02  0.022 0.025 0.026 0.023 0.032 0.024]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
610
[-226853.7 -213830.5 -515874.1]
[[-1.117 -0.177 -0.697 -0.891  0.161 -0.063  0.133  0.116  0.148 -0.205
  -1.176 -0.304  0.848 -0.805  0.009 -0.887]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146 

800
[-224747.6 -213830.5 -515874.1]
[[-1.141 -0.164 -0.691 -0.897  0.144 -0.066  0.138  0.116  0.156 -0.191
  -1.183 -0.289  0.845 -0.828  0.006 -0.913]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.053 0.025 0.025 0.021 0.022 0.024 0.027 0.022 0.038 0.022]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
810
[-224481.9 -213830.5 -515874.1]
[[-1.148 -0.16  -0.703 -0.9    0.156 -0.047  0.135  0.114  0.14  -0.194
  -1.184 -0.295  0.848 -0.825  0.004 -0.906]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.061 0.028 0.024 0.023 0.021 0.02  0.026 0.023 0.037 0.022]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
820
[-224298.1 -213830.5 -515874.1]
[[-1.147 -0.158 -0.701 -0.887  0.144 -0.064  0.136  0.111  0.151 -0.198
  -1.174 -0.291  0.847 -0.827 -0.012 -0.914]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146 

1010
[-222318.3 -213830.5 -515874.1]
[[-1.173 -0.172 -0.705 -0.887  0.151 -0.064  0.128  0.116  0.154 -0.201
  -1.178 -0.292  0.841 -0.823 -0.006 -0.906]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.057 0.026 0.025 0.025 0.022 0.022 0.026 0.021 0.05  0.025]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1020
[-222572.3 -213830.5 -515874.1]
[[-1.167 -0.17  -0.703 -0.888  0.153 -0.068  0.138  0.122  0.15  -0.187
  -1.176 -0.3    0.829 -0.833 -0.013 -0.918]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.057 0.028 0.026 0.022 0.022 0.021 0.025 0.021 0.045 0.025]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1030
[-222219.1 -213830.5 -515874.1]
[[-1.198 -0.163 -0.694 -0.888  0.164 -0.06   0.143  0.123  0.15  -0.197
  -1.18  -0.3    0.839 -0.826  0.006 -0.905]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.1

1220
[-222357.8 -213830.5 -515874.1]
[[-1.157 -0.167 -0.705 -0.888  0.142 -0.066  0.139  0.112  0.146 -0.192
  -1.181 -0.301  0.835 -0.849 -0.015 -0.916]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.055 0.026 0.025 0.023 0.022 0.025 0.024 0.022 0.04  0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1230
[-222522.3 -213830.5 -515874.1]
[[-1.172 -0.171 -0.707 -0.894  0.157 -0.066  0.136  0.125  0.154 -0.192
  -1.181 -0.3    0.844 -0.825 -0.002 -0.915]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.051 0.029 0.023 0.021 0.02  0.025 0.024 0.024 0.038 0.022]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1240
[-222287.2 -213830.5 -515874.1]
[[-1.184 -0.17  -0.701 -0.889  0.148 -0.068  0.135  0.122  0.158 -0.194
  -1.182 -0.296  0.851 -0.821  0.008 -0.9  ]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.1

1420
[-222614.1 -213830.5 -515874.1]
[[-1.158 -0.166 -0.705 -0.89   0.144 -0.06   0.131  0.119  0.154 -0.206
  -1.185 -0.304  0.842 -0.828  0.005 -0.919]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.049 0.024 0.025 0.022 0.022 0.023 0.028 0.025 0.046 0.023]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1430
[-221992.7 -213830.5 -515874.1]
[[-1.171 -0.169 -0.715 -0.902  0.149 -0.067  0.141  0.121  0.152 -0.194
  -1.178 -0.3    0.846 -0.836  0.003 -0.906]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.053 0.022 0.022 0.02  0.022 0.021 0.024 0.022 0.045 0.024]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1440
[-221719.  -213830.5 -515874.1]
[[-1.198 -0.173 -0.688 -0.885  0.136 -0.053  0.133  0.123  0.157 -0.198
  -1.176 -0.296  0.841 -0.823  0.004 -0.905]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.1

1620
[-220922.2 -213830.5 -515874.1]
[[-1.189 -0.177 -0.694 -0.883  0.152 -0.076  0.132  0.112  0.154 -0.196
  -1.186 -0.306  0.838 -0.831 -0.003 -0.902]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.056 0.026 0.023 0.023 0.022 0.024 0.024 0.02  0.045 0.02 ]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1630
[-220612.3 -213830.5 -515874.1]
[[-1.205 -0.18  -0.69  -0.89   0.155 -0.054  0.132  0.116  0.167 -0.199
  -1.177 -0.299  0.829 -0.839 -0.024 -0.918]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.06  0.025 0.021 0.022 0.023 0.024 0.024 0.021 0.048 0.021]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1640
[-220877.6 -213830.5 -515874.1]
[[-1.207 -0.171 -0.691 -0.9    0.156 -0.063  0.139  0.117  0.159 -0.201
  -1.175 -0.298  0.841 -0.825 -0.005 -0.91 ]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.1

1810
[-221023.9 -213830.5 -515874.1]
[[-1.201 -0.163 -0.7   -0.899  0.153 -0.069  0.141  0.122  0.155 -0.2
  -1.173 -0.299  0.838 -0.833 -0.011 -0.904]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.045 0.023 0.024 0.02  0.022 0.022 0.027 0.022 0.049 0.025]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1820
[-220881.9 -213830.5 -515874.1]
[[-1.21  -0.167 -0.702 -0.902  0.158 -0.06   0.141  0.119  0.155 -0.197
  -1.178 -0.292  0.845 -0.827  0.003 -0.904]
 [-1.3   -0.184 -0.775 -0.981  0.17  -0.071  0.146  0.129  0.171 -0.221
  -1.305 -0.338  0.934 -0.921 -0.023 -1.01 ]]
[[0.048 0.025 0.022 0.022 0.021 0.023 0.026 0.02  0.052 0.022]
 [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025]]
1830
[-220843.8 -213830.5 -515874.1]
[[-1.209e+00 -1.790e-01 -6.970e-01 -8.930e-01  1.550e-01 -5.300e-02
   1.350e-01  1.170e-01  1.580e-01 -1.950e-01 -1.178e+00 -2.990e-01
   8.400e-01 -8.260e-01 -1.000e